### Transfer Training

모델 설계  

- 특징 추출 부분(초기 Conv 레이어들)을 동결하고, 출력층(FC Layer)만 재학습
- AdaptiveAvgPool2d

1) 텐서의 공간 차원(Height, Width)을 평균 연산으로 고정된 크기로 변환.
2) 공간 정보를 축소하거나, CNN 출력 크기를 Fully Connected Layer 입력으로 변환하기 전에 사용.
3) 예: 입력 크기가 (Batch, Channels, Height, Width)일 때 Height와 Width를 원하는 고정 값으로 줄임.

In [8]:
# import libraries
import os
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

import time
import copy

In [9]:
# move model to GPU if available
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

# batch size, epoch
BATCH_SIZE = 256
EPOCH = 30

cuda


In [10]:
# resnet = models.vgg16(pretrained=True)
resnet = models.resnet152(pretrained=True)
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
# image to tensor (64 * 64)
transform_base = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])

train_dataset = ImageFolder(root='./splitted/train/', transform=transform_base)
val_dataset = ImageFolder(root='./splitted/val/', transform=transform_base)

In [12]:
# DataLoader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [ ]:
# model
class Net(nn.Module) :

    def __init__(self) :
        super(Net, self).__init__()
        self.resnet = models.resnet152(pretrained=True)

        for param in self.resnet.parameters():
            param.requires_grad = False

        cnt = self.resnet.fc.in_features

        self.resnet.fc = nn.Linear(in_features=cnt, out_features=33)

    def forward(self, x) :
        x = self.resnet(x)
        return F.log_softmax(x, dim=1)

In [14]:
# create model
model_base = Net().to(DEVICE)

# gradient descent
optimizer = optim.Adam(model_base.parameters(), lr=0.001)

In [15]:
# training function
def train(model, train_loader, optimizer) :
    model.train()
    
    for data, target in train_loader :
        data = data.to(DEVICE)
        target = target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [16]:
# evaluation function
def evaluate(model, test_loader) :
    model.eval()
    test_loss = 0
    correct = 0

    ## not working gradient descent
    with torch.no_grad():
        for data, target in test_loader :
            data = data.to(DEVICE)
            target = target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    # average of loss
    test_loss /= len(test_loader.dataset)
    # correct probability
    test_accuracy = 100.0 * correct / len(test_loader.dataset)

    return test_loss, test_accuracy

In [17]:
# train_baseline func
def train_baseline(model, train_loader, val_loader, optimizer, num_epochs=30) :
    best_acc = 0
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(1, num_epochs + 1):
        since = time.time()
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader)
        val_loss, val_acc = evaluate(model, val_loader)

        if val_acc > best_acc :
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
                    
        time_elapsed = time.time() - since
        
        print(f'Train number [{epoch}]')
        print(f'[Train] - Loss  {train_loss} | Accuracy: {train_acc}')
        print(f'[Validation] - Loss  {val_loss} | Validation: {val_acc}')
        print(f'Train Time: {time_elapsed}')
        print('--------------')

    # after training, best weight model
    model.load_state_dict(best_model_wts)
    return model

In [18]:
# train
# base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)
base = train_baseline(model_base, train_loader, val_loader, optimizer, 2)
torch.save(base, 'resnet152.pt')

Train number [1]
[Train] - Loss  0.7864441510926874 | Accuracy: 78.83049592894153
[Validation] - Loss  0.8591111675404253 | Validation: 76.79768433881779
Train Time: 35.84887480735779
--------------
Train number [2]
[Train] - Loss  0.6108012557294438 | Accuracy: 83.08413520848754
[Validation] - Loss  0.7321220365404574 | Validation: 79.0219378427788
Train Time: 33.4819815158844
--------------


In [19]:
# predict
test_dataset = ImageFolder(root='./splitted/test/', transform=transform_base)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
data_iter = iter(test_loader)
batch_x, batch_y = next(data_iter)

print("- Feature (x):", batch_x)
print("- Label (y):", batch_y)
print("- Feature shape:", batch_x.shape)
print("- Label shape:", batch_y.shape)

- Feature (x): tensor([[[[0.4588, 0.4667, 0.4784,  ..., 0.4902, 0.5098, 0.4941],
          [0.4784, 0.4706, 0.4745,  ..., 0.5098, 0.5137, 0.5020],
          [0.4667, 0.4706, 0.4627,  ..., 0.4941, 0.4980, 0.5020],
          ...,
          [0.4549, 0.4588, 0.4588,  ..., 0.4510, 0.4510, 0.4510],
          [0.4588, 0.4549, 0.4588,  ..., 0.4431, 0.4431, 0.4392],
          [0.4588, 0.4549, 0.4549,  ..., 0.4157, 0.4196, 0.4118]],

         [[0.4431, 0.4510, 0.4627,  ..., 0.4706, 0.4902, 0.4745],
          [0.4627, 0.4549, 0.4588,  ..., 0.4902, 0.4941, 0.4824],
          [0.4510, 0.4549, 0.4471,  ..., 0.4745, 0.4784, 0.4824],
          ...,
          [0.4353, 0.4392, 0.4392,  ..., 0.4235, 0.4275, 0.4275],
          [0.4392, 0.4353, 0.4392,  ..., 0.4196, 0.4196, 0.4157],
          [0.4392, 0.4353, 0.4353,  ..., 0.3961, 0.4000, 0.3922]],

         [[0.5725, 0.5804, 0.5922,  ..., 0.5961, 0.6157, 0.6000],
          [0.5922, 0.5843, 0.5882,  ..., 0.6157, 0.6196, 0.6078],
          [0.5804, 0.5843, 

In [20]:
# predict
test_dataset = ImageFolder(root='./splitted/test/', transform=transform_base)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

model = torch.load('resnet152.pt')

with torch.no_grad() :
    result = []

    for data, target in test_loader :
        data = data.to(DEVICE)
        target = target.to(DEVICE)
        output = model(data)

        pred = output.max(1, keepdim=True)[1]
        pred = pred.cpu().numpy()
        result = result + pred.reshape(-1).tolist()

print(result)

[23, 23, 5, 4, 9, 3, 7, 12, 19, 9, 14, 28, 8, 16, 11, 32, 27, 8, 21, 10, 3, 22, 28, 14, 19, 8, 25, 23, 24, 10, 27, 26, 27, 10, 7, 21, 14, 22, 14, 4, 23, 0, 19, 14, 23, 0, 4, 28, 8, 7, 4, 27, 18, 21, 9, 25, 32, 18, 25, 13, 28, 10, 19, 19, 32, 29, 10, 28, 1, 9, 10, 18, 23, 12, 3, 28, 13, 17, 11, 19, 12, 7, 5, 25, 26, 25, 23, 17, 5, 4, 23, 16, 3, 27, 8, 29, 32, 14, 32, 23, 8, 10, 19, 16, 23, 12, 12, 10, 8, 16, 22, 23, 17, 2, 16, 4, 0, 25, 28, 21, 3, 28, 10, 3, 7, 6, 23, 0, 18, 21, 23, 4, 9, 28, 4, 20, 2, 16, 17, 3, 18, 8, 32, 17, 22, 21, 26, 28, 5, 11, 14, 17, 32, 19, 17, 4, 28, 25, 23, 27, 3, 24, 17, 21, 16, 23, 23, 32, 11, 2, 25, 15, 14, 3, 24, 25, 4, 10, 4, 22, 8, 17, 12, 27, 13, 5, 7, 32, 9, 27, 32, 11, 15, 17, 9, 12, 28, 28, 7, 32, 1, 25, 27, 0, 10, 24, 26, 9, 14, 28, 3, 9, 9, 24, 23, 28, 8, 25, 1, 17, 23, 6, 32, 27, 27, 25, 0, 10, 4, 21, 21, 26, 21, 23, 7, 23, 21, 4, 22, 25, 10, 29, 9, 18, 9, 27, 6, 18, 23, 7, 14, 17, 8, 18, 28, 21, 27, 4, 12, 15, 14, 32, 28, 3, 6, 23, 8, 21, 15, 19

In [21]:
results_data = []
classes_dict = test_dataset.class_to_idx
classes_names = list(classes_dict.keys())

for r1 in result :
    n1 = classes_names[r1]
    results_data.append(n1)

print(results_data)

['Tomato___Bacterial_spot', 'Tomato___Bacterial_spot', 'Cherry___healthy', 'Cherry___Powdery_mildew', 'Corn___healthy', 'Apple___healthy', 'Corn___Common_rust', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Potato___Late_blight', 'Corn___healthy', 'Peach___Bacterial_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Corn___Northern_Leaf_Blight', 'Pepper,_bell___Bacterial_spot', 'Grape___Esca_(Black_Measles)', 'Tomato___healthy', 'Tomato___Septoria_leaf_spot', 'Corn___Northern_Leaf_Blight', 'Strawberry___Leaf_scorch', 'Grape___Black_rot', 'Apple___healthy', 'Strawberry___healthy', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Peach___Bacterial_spot', 'Potato___Late_blight', 'Corn___Northern_Leaf_Blight', 'Tomato___Late_blight', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Grape___Black_rot', 'Tomato___Septoria_leaf_spot', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Grape___Black_rot', 'Corn___Common_rust', 'Strawberry___Leaf_scorch', 'Peach___Bacterial_spot'